In [ ]:
import pandas as pd

# Import csv files names
records_output_csv = "records_data.csv"
workouts_output_csv = "workouts_data.csv"

records = pd.read_csv(records_output_csv)
records

In [ ]:
workouts = pd.read_csv(workouts_output_csv)
workouts

In [5]:
def get_records_for_workout(
    all_records, record_type, workout_startdate, workout_enddate
):
    """Get details of a record type for a workout"""
    records = all_records.loc[all_records["type"] == record_type]
    records = records.loc[
        (records["startDate"] >= workout_startdate)
        & (records["endDate"] <= workout_enddate)
    ]
    if not records.empty:
        records.reset_index(drop=True, inplace=True)
        unit = records["unit"].iloc[0]
        if record_type in [
            "HeartRate",
            "RunningStrideLength",
            "RunningPower",
            "RunningVerticalOscillation",
            "RunningGroundContactTime",
            "RunningSpeed",
        ]:
            return {
                f"max {record_type} ({unit})": records["value"].max(),
                f"min {record_type} ({unit})": records["value"].min(),
                f"mean {record_type} ({unit})": records["value"].mean(),
            }
        if record_type in ["ActiveEnergyBurned", "BasalEnergyBurned"]:
            return {f"{record_type} ({unit})": records["value"].sum()}
    return {}

In [4]:
def get_workout_statistics(all_workouts, workout_type, all_records, record_types):
    """Get workout statisticas"""
    workouts = all_workouts.loc[all_workouts["workoutActivityType"] == workout_type]
    statistics = []
    for index, row in workouts.iterrows():
        statistic = {}
        statistic["type"] = workout_type
        statistic[f'duration ({row["durationUnit"]})'] = row["duration"]
        statistic["time"] = row["startDate"]
        statistic["date"] = row["startDate"][5:10]
        for record_type in record_types:
            statistic.update(
                get_records_for_workout(
                    all_records, record_type, row["startDate"], row["endDate"]
                )
            )
        if len(statistic) == 24:
                pace = 1000 / 60 / statistic["mean RunningSpeed (m/s)"]
                statistic["pace (min/km)"] = f"{pace:.2f}"
                statistic[
                    "distance (km)"
                ] = f'{(statistic["duration (min)"] / pace):.2f}'
                if float(statistic["distance (km)"]) >= 5.0:
                    statistics.append(statistic)
    return statistics

In [ ]:
get_workout_statistics(workouts, "Running", records, ["HeartRate", "RunningStrideLength", "RunningPower", "RunningVerticalOscillation", "RunningGroundContactTime", "RunningSpeed", "ActiveEnergyBurned", "BasalEnergyBurned"])